# Disease Diagnose

Nosso trabalho consiste em auxiliar o médico na hora de diagnosticar seus
pacientes.  
Através de um banco de dados, que será alimentado, nosso sistema irá indicar
probabilidades de diagnósticos, ou seja, qual doença é mais possível que o paciente tenha
contraído.  
Após a análise o médico poderá avaliar, ou pedir um exame, para confirmar o
diagnóstico. Caso essa avaliação não confirme a doença com mais probabilidade o médico
poderá alimentar o sistema com essa informação e permitir com que as probabilidades
sejam recalculadas a partir daquela informação.  


## Modelo Conceitual 

<img src="UML - Etapa02-Page-1.png">

# Modelagem Lógica 


In [2]:
%defaultDatasource jdbc:h2:mem:db


In [3]:
CREATE TABLE Paciente (
  ID VARCHAR(9) NOT NULL,
  nome VARCHAR(50),
  nascimento VARCHAR(10),
  celular VARCHAR(7),
  localidade VARCHAR(9),
  PRIMARY KEY(ID)
) AS SELECT
    ID,
    nome,
    nascimento,
    celular, 
    localidade
FROM CSVREAD('../data/Paciente.csv');

CREATE TABLE Doenca (
  codigo VARCHAR(8) NOT NULL,
  nome VARCHAR(50),
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Doenca.csv');

CREATE TABLE Sintomas (
  codigo VARCHAR(8) NOT NULL,
  nome VARCHAR(50),
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Sintomas.csv');



In [5]:
--INSERT INTO Paciente VALUES('pat12345','Joao','11/08/1995','3234589','Northeast');
INSERT INTO Doenca VALUES('afk12345','Gripe');
INSERT INTO Sintomas VALUES('hrg56789','coceira');

org.h2.jdbc.JdbcSQLException:  Unique index or primary key violation

In [6]:
CREATE TABLE Diagnostico (
  ID VARCHAR(10) NOT NULL,
  doencaCodigo VARCHAR(8),
  data VARCHAR(10),
  PRIMARY KEY(ID),
  FOREIGN KEY(doencaCodigo)
    REFERENCES Doenca(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    ID,
    doencaCodigo,
    data
FROM CSVREAD('../data/Diagnostico.csv');

CREATE TABLE DoencaSintomas (
  doencaCodigo VARCHAR(8) NOT NULL,
  sintomaCodigo VARCHAR(8) NOT NULL,
  FOREIGN KEY(doencaCodigo)
    REFERENCES Doenca(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(sintomaCodigo)
    REFERENCES Sintomas(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    doencaCodigo,
    sintomaCodigo
FROM CSVREAD('../data/DoencaSintomas.csv');



In [7]:
INSERT INTO Diagnostico VALUES ('123456789A','afk12345','10/04/1997');
INSERT INTO DoencaSintomas VALUES ('afk12345','hrg56789');

In [8]:
CREATE TABLE PacienteDiagnostico (
  pacienteID VARCHAR(9) NOT NULL,
  diagnosticoID VARCHAR(10) NOT NULL,
  FOREIGN KEY(pacienteID)
    REFERENCES Paciente(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(diagnosticoID)
    REFERENCES Diagnostico(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    pacienteID,
    diagnosticoID
FROM CSVREAD('../data/PacienteDiagnostico.csv');


In [ ]:

CREATE TABLE PacienteSintomas (
  pacienteID VARCHAR(9) NOT NULL,
  sintomaCodigo VARCHAR(8) NOT NULL,
  FOREIGN KEY(pacienteID)
    REFERENCES Paciente(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(sintomaCodigo)
    REFERENCES Sintomas(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    pacienteID,
    sintomaCodigo
FROM CSVREAD('../data/PacienteSintomas.csv');




In [14]:
CREATE TABLE Educacao (
  Dcode VARCHAR(8) NOT NULL,
  Educacao VARCHAR(50) NOT NULL,
  Prob FLOAT(4,2),
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Educacao, 
    Prob
FROM CSVREAD('../data/Educacao.csv');

CREATE TABLE Emprego (
  Dcode VARCHAR(8) NOT NULL,
  Educacao VARCHAR(30),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcodigo,
    Emprego, 
    Prob
FROM CSVREAD('../data/Emprego.csv');

CREATE TABLE Idade (
  Dcode VARCHAR(8) NOT NULL,
  Educacao VARCHAR(30),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Idade, 
    Prob
FROM CSVREAD('../data/Idade.csv');

CREATE TABLE Raca (
  D_code VARCHAR(8) NOT NULL,
  Raca VARCHAR(30),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Raça, 
    Prob
FROM CSVREAD('../data/Raca.csv');

CREATE TABLE Regiao (
  Dcode VARCHAR(8) NOT NULL,
  Regiao VARCHAR(30),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Região, 
    Prob
FROM CSVREAD('../data/Regiao.csv');

CREATE TABLE Renda (
  Dcode VARCHAR(8) NOT NULL,
  Renda VARCHAR(30),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Renda, 
    Prob
FROM CSVREAD('../data/Renda.csv');

CREATE TABLE Sexo (
  Dcode VARCHAR(8) NOT NULL,
  Sexo VARCHAR(1),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Sexo, 
    Prob
FROM CSVREAD('../data/Sexo.csv');

CREATE TABLE Status (
  Dcode VARCHAR(8) NOT NULL,
  Status VARCHAR(1),
  Prob FLOAT(4,2)
  PRIMARY KEY(Dcode)
) AS SELECT
    Dcode,
    Status, 
    Prob
FROM CSVREAD('../data/Status.csv');


org.h2.jdbc.JdbcSQLException:  Syntax error in SQL statement "CREATE TABLE EDUCACAO (

## Queries

In [9]:
SELECT * FROM Paciente WHERE Paciente.nascimento LIKE '%1990'

# Sintoma mais relatado por pacientes

In [10]:
SELECT nome FROM (SELECT S.nome, COUNT(*) contagem FROM Sintomas S, DoencaSintomas DS, Doenca D WHERE DS.sintomaCodigo = S.codigo AND DS.doencaCodigo = D.codigo GROUP BY S.nome)
WHERE contagem = (SELECT MAX(contagem) FROM (SELECT S.nome, COUNT(*) contagem FROM Sintomas S, DoencaSintomas DS, Doenca D WHERE DS.sintomaCodigo = S.codigo AND DS.doencaCodigo = D.codigo GROUP BY S.nome))

shortness of breath

# Nome dos constatados com a doença X (no caso abaixo: diabetes)

In [11]:
SELECT P.nome FROM PacienteDiagnostico PD, Diagnostico D, Doenca Do, Paciente P WHERE PD.pacienteID = P.ID AND PD.diagnosticoID = D.ID AND D.doencaCodigo = Do.codigo AND Do.nome = 'diabetes'

In [13]:
SELECT D.nome, COUNT(*) ocorrencias FROM Diagnostico Di, Doenca D, Sintomas S WHERE D.codigo = Di.doencaCodigo AND GROUP BY D.nome; 